In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/media-sequence/media_sequence.csv
/kaggle/input/dsb-media-sequence/media_sequence.csv
/kaggle/input/data-science-bowl-2019/train.csv
/kaggle/input/data-science-bowl-2019/train_labels.csv
/kaggle/input/data-science-bowl-2019/specs.csv
/kaggle/input/data-science-bowl-2019/test.csv
/kaggle/input/data-science-bowl-2019/sample_submission.csv
/kaggle/input/null-importance/2020_2_2_null_importance.csv


### top 700 + add test label

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt 
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
import catboost as cb
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, ShuffleSplit,StratifiedKFold,TimeSeriesSplit,KFold,GroupKFold,train_test_split,GroupShuffleSplit,StratifiedShuffleSplit
from sklearn.metrics import roc_auc_score,mean_squared_error,mean_absolute_error,log_loss,confusion_matrix
import sqlite3
import xgboost as xgb
import datetime
from sklearn.linear_model import LogisticRegression
from scipy.stats import pearsonr
import gc
from sklearn.model_selection import TimeSeriesSplit
from bayes_opt import BayesianOptimization
import re
from string import punctuation
from scipy.spatial import Voronoi
from scipy.spatial import ConvexHull
from scipy.spatial import Delaunay
from tqdm import tqdm
from numba import jit
from collections import Counter
import json

In [3]:
def to_accuracy_group(accuracy):
    if accuracy ==0:
        return 0
    elif accuracy ==1:
        return 3
    elif accuracy ==0.5:
        return 2
    else:
        return 1
@jit
def qwk(a1, a2):
    """
    Source: https://www.kaggle.com/c/data-science-bowl-2019/discussion/114133#latest-660168

    :param a1:
    :param a2:
    :param max_rat:
    :return:
    """
    max_rat = 3
    a1 = np.asarray(a1, dtype=int)
    a2 = np.asarray(a2, dtype=int)

    hist1 = np.zeros((max_rat + 1, ))
    hist2 = np.zeros((max_rat + 1, ))

    o = 0
    for k in range(a1.shape[0]):
        i, j = a1[k], a2[k]
        hist1[i] += 1
        hist2[j] += 1
        o +=  (i - j) * (i - j)

    e = 0
    for i in range(max_rat + 1):
        for j in range(max_rat + 1):
            e += hist1[i] * hist2[j] * (i - j) * (i - j)

    e = e / a1.shape[0]

    return 1 - o / e

In [4]:
def get_kappa(y_pred,y_true,coefficients):
    preds = y_pred.copy()
    preds[y_pred <= coefficients[0]] = 0
    preds[np.where(np.logical_and(y_pred > coefficients[0], y_pred <= coefficients[1]))] = 1
    preds[np.where(np.logical_and(y_pred > coefficients[1], y_pred <= coefficients[2]))] = 2
    preds[y_pred > coefficients[2]] = 3
    return qwk(y_true,preds)

def get_pred(y_pred,coefficients):
    preds = y_pred.copy()
    preds[y_pred <= coefficients[0]] = 0
    preds[np.where(np.logical_and(y_pred > coefficients[0], y_pred <= coefficients[1]))] = 1
    preds[np.where(np.logical_and(y_pred > coefficients[1], y_pred <= coefficients[2]))] = 2
    preds[y_pred > coefficients[2]] = 3
    return preds

def opt_more_coeff(oof_predict,y_train,coef):
    score1 = 0
    k = 0
    for a in np.arange(-0.1,0.1,0.01):
        for b in np.arange(-0.1,0.1,0.01):
            for c in np.arange(-0.1,0.1,0.01):
                coefficients = [coef[0]+a, coef[1]+b, coef[2]+c]
                preds = oof_predict.copy()
                preds[oof_predict <= coefficients[0]] = 0
                preds[np.where(np.logical_and(oof_predict > coefficients[0], oof_predict <= coefficients[1]))] = 1
                preds[np.where(np.logical_and(oof_predict > coefficients[1], oof_predict <= coefficients[2]))] = 2
                preds[oof_predict > coefficients[2]] = 3
                kappa_1 = qwk(y_train, preds)
                k+=1
                if kappa_1>score1:
                    temp_save = [coef[0]+a, coef[1]+b, coef[2]+c]
                    #print(np.round(kappa_1,8),'Per:',str(round((k/64000)*100,2))+'%',k,temp_save)
                    score1 = kappa_1
    coef = temp_save.copy()
    ### second 
    k=0
    for a in np.arange(-0.04,0.04,0.004):
        for b in np.arange(-0.04,0.04,0.004):
            for c in np.arange(-0.04,0.04,0.004):
                coefficients = [coef[0]+a, coef[1]+b, coef[2]+c]
                preds = oof_predict.copy()
                preds[oof_predict <= coefficients[0]] = 0
                preds[np.where(np.logical_and(oof_predict > coefficients[0], oof_predict <= coefficients[1]))] = 1
                preds[np.where(np.logical_and(oof_predict > coefficients[1], oof_predict <= coefficients[2]))] = 2
                preds[oof_predict > coefficients[2]] = 3
                kappa_1 = qwk(y_train, preds)
                k+=1
                if kappa_1>score1:
                    temp_save = [coef[0]+a, coef[1]+b, coef[2]+c]
                    #print(np.round(kappa_1,8),'Per:',str(round((k/64000)*100,2))+'%',k,temp_save)
                    score1 = kappa_1
    return temp_save,score1

In [5]:
train = pd.read_csv('/kaggle/input/data-science-bowl-2019/train.csv')
specs = pd.read_csv('/kaggle/input/data-science-bowl-2019/specs.csv')
test = pd.read_csv('/kaggle/input/data-science-bowl-2019/test.csv')
train_labels = pd.read_csv('/kaggle/input/data-science-bowl-2019/train_labels.csv')
sample_submission = pd.read_csv('/kaggle/input/data-science-bowl-2019/sample_submission.csv')
train['timestamp'] = pd.to_datetime(train.timestamp)
test['timestamp'] = pd.to_datetime(test.timestamp)
null_importance = pd.read_csv('/kaggle/input/null-importance/2020_2_2_null_importance.csv')

In [6]:
orders = null_importance.iloc[:,0].tolist()

In [7]:
train['title_event_code'] = train['title'].astype(str) + '_' + train['event_code'].astype(str)
test['title_event_code'] = test['title'].astype(str) + '_' + test['event_code'].astype(str)

In [8]:
list_of_event_code = set(train.event_code).union(set(test.event_code))
list_of_event_id = set(train.event_id).union(set(test.event_id))
list_title = set(train.title).union(set(test.title))
list_title_event_code = set(train.title_event_code).union(set(test.title_event_code))

In [9]:
def voting(test_sample):
    pred = [] 
    for i in range(test_sample.shape[1]):
        result = test_sample[:,i]
        value_counts = pd.Series(result).value_counts()
        pred_value = np.random.permutation(list(value_counts.index[value_counts==value_counts.iloc[0]]))[0]
        pred.append(pred_value)
    return np.array(pred)

In [10]:
def lr_decay(index_):
    if index_ < 20:
        return 0.0003
    elif  index_ < 40:
        if  index_ % 2 ==0:
            return 0.00008
        else:
            return 0.0002                    
    elif  index_ < 60:
        if  index_ % 2 ==0:
            return 0.00008
        else:
            return 0.00003          
    else:
        return 0.00003

In [11]:
from keras.layers import LSTM, Dropout, Input, Dense, concatenate,Flatten, BatchNormalization, Softmax, Add,LeakyReLU
from keras.models import Model,Sequential,load_model
from keras import optimizers
from keras_tqdm import TQDMNotebookCallback
from sklearn.model_selection import train_test_split
from  keras.callbacks import EarlyStopping,ModelCheckpoint
import keras


def NN_model():
    Input_main = Input(shape=((X_train.shape[1],)), name="main")
    main_input = Dense(256)(Input_main)
    main_input = LeakyReLU(alpha=0.3)(main_input)
    main_input = BatchNormalization()(main_input)
    main_input = Dropout(0.3)(main_input) 
    
    main_input = Dense(256)(main_input)
    main_input = LeakyReLU(alpha=0.3)(main_input)
    main_input = BatchNormalization()(main_input)   
    main_input = Dropout(0.3)(main_input)    

    main_input = Dense(256)(main_input)    
    main_input = LeakyReLU(alpha=0.3)(main_input)
    main_input = BatchNormalization()(main_input)
    main_input = Dropout(0.3)(main_input)   
    
    main_input = Dense(128)(main_input)    
    main_input = LeakyReLU(alpha=0.3)(main_input)
    main_input = BatchNormalization()(main_input)
    main_input = Dropout(0.3)(main_input) 


    output = Dense(1)(main_input)
    output2 = Dense(1)(main_input)
    adam = optimizers.Adam(lr=0.0003, beta_1=0.9, beta_2=0.999, amsgrad=False)
    model = Model(inputs=[Input_main], outputs=[output])
    model.compile(optimizer=adam, loss=['mean_squared_error'])
    return model



Using TensorFlow backend.


In [12]:
def normal_nn(X_train,y_train,ins_id,X_test):
    coefficients = [1.051, 1.682, 2.231]
    gkf = GroupKFold(n_splits=4)
    impor1 = 0
    score = 0
    score_kappa = 0
    models = []
    oof_predict = np.zeros((y_train.shape[0]))
    pred_test1 = 0
    for train_index, test_index in gkf.split(X_train, y_train,groups=ins_id):
        X_train2= X_train.iloc[train_index,:]
        y_train2= y_train.iloc[train_index]
        X_test2= X_train.iloc[test_index,:]
        y_test2= y_train.iloc[test_index]
        clf = NN_model()
        es = EarlyStopping(monitor='val_loss', patience=30, verbose=2,restore_best_weights=True)
        lr = keras.callbacks.LearningRateScheduler(lr_decay)
        clf.fit(X_train2, y_train2,validation_data=(X_test2,y_test2),callbacks=[es,lr],batch_size=128,epochs=300,verbose=2)
        models.append(clf)
        temp_predict = clf.predict(X_test2).flatten()
        pred_test1 +=  clf.predict(X_test).flatten()/gkf.n_splits
        oof_predict[test_index] = temp_predict
        rse_1 = np.sqrt(mean_squared_error(y_test2,temp_predict))
        print('rse:',rse_1)
        preds = temp_predict.copy()
        preds[preds <= coefficients[0]] = 0
        preds[np.where(np.logical_and(preds > coefficients[0], preds <= coefficients[1]))] = 1
        preds[np.where(np.logical_and(preds > coefficients[1], preds <= coefficients[2]))] = 2
        preds[preds > coefficients[2]] = 3
        kappa_1 = qwk(y_test2, preds)
        score += rse_1/gkf.n_splits
        score_kappa += kappa_1/gkf.n_splits
        gc.collect()
    print('mean rse:',score)
    print('mean kappa:',score_kappa)
    preds = oof_predict.copy()
    preds[preds <= coefficients[0]] = 0
    preds[np.where(np.logical_and(preds > coefficients[0], preds <= coefficients[1]))] = 1
    preds[np.where(np.logical_and(preds > coefficients[1], preds <= coefficients[2]))] = 2
    preds[preds > coefficients[2]] = 3
    print('oof kappa:',qwk(y_train, preds))
    return oof_predict,preds,pred_test1,score

In [13]:
import warnings
warnings.filterwarnings('ignore')

In [14]:
def run_all_nn(X_train,y_train,ins_id,X_test):
    cappa_list = []
    oof_class_list = []
    rmse_list = []
    pred_class_list = []
    oof_num_mean = 0
    pred_num_pred = 0
    temp_score_overall = []
    
    seed_list = [7,8,9,10,11]
    for kth,seed1 in enumerate(seed_list):#[32,42,47,1103,128,22,45][11,22,33,43,45,56,67]
        print(str(kth+1),'th seed begins----------------------' )
        oof_num,oof_class,pred_num,rmse = normal_nn(X_train,y_train,ins_id,X_test)
        pred_num_pred += pred_num/len(seed_list)
        coefficients = [1.051, 1.682, 2.231]
        temp_inter = 0
        temp_score = 0 
        
        for i in range(400):###100可以继续往上加
            t,score = opt_more_coeff(oof_num[list(truncated_groups[:,i*2+seed1])],y_train.loc[list(truncated_groups[:,i*2+seed1])],[1.051, 1.682, 2.231])

            temp_inter +=np.array(t)/400
            temp_score +=score/400
        print(temp_inter)
        print('mean cappa',temp_score)
        temp_score_overall.append(temp_score)
        
        coefficients2 = list(temp_inter)
        print('New threshold',coefficients2)
        y_pred_class = get_pred(pred_num,coefficients2)
        print(pd.Series(y_pred_class).value_counts())
        cappa1 = qwk(y_train,get_pred(oof_num,coefficients2))
        cappa_list.append(cappa1)
        oof_class = get_pred(oof_num,coefficients2)
        oof_class_list.append(oof_class)
        oof_num_mean += oof_num/len(seed_list)
        rmse_list.append(rmse)
        pred_class_list.append(y_pred_class)
    print('Overall situation-----------------------')
    print('Mean rmse {}'.format(round(np.mean(rmse_list),5)))
    print('Mean cappa {}'.format(round(np.mean(temp_score_overall),5)))
    impor2 = 0

    return voting(np.vstack(tuple(pred_class_list))),impor2,voting(np.vstack(tuple(oof_class_list))),oof_num_mean,pred_num_pred


In [15]:
clip_time = pd.read_csv('/kaggle/input/dsb-media-sequence/media_sequence.csv')
clip_time = clip_time[clip_time.type == 'Clip']
clip_time= clip_time[['title','duration']].set_index('title')['duration']

In [16]:
assessment_list = ['Cart Balancer (Assessment)',
 'Cauldron Filler (Assessment)',
 'Mushroom Sorter (Assessment)',
 'Chest Sorter (Assessment)',
 'Bird Measurer (Assessment)']

world_list = ['CRYSTALCAVES','MAGMAPEAK','TREETOPCITY']


duration_event_id = ['90efca10', '4ef8cdd3', '9ee1c98c', 'e694a35b', '08fd73f3', '2dc29e21',
       '499edb7c', 'e9c52111', 'd185d3ea', 'd3f1e122', 'cdd22e43', 'fcfdffb6',
       '84538528', 'c58186bf', 'de26c3a6', '5c2f29ca', '598f4598', '804ee27f',
       '56817e2b', '37c53127', '4a4c3d21', '363c86c9', 'e7561dd2', '022b4259',
       '5a848010', '562cec5f', '71e712d8', 'd02b7a8e', '30614231', '3323d7e9',
       '5f0eb72c', '0db6d71d', '4a09ace1', 'ca11f653', '1c178d24', '46cd75b4',
       '00c73085', 'bc8f2793', '36fa3ebe', '16dffff1', '8fee50e2', '392e14df',
       '9d4e7b25', 'd06f75b5', '895865f3', 'c1cac9a2', '28520915', 'd51b1749',
       'b012cd7f', 'f5b8c21a', '3d0b9317', '9ce586dd', '3d63345e', 'c7128948',
       '5348fd84', '83c6c409', '99abe2bb', 'c189aaf2', '99ea62f3', 'b74258a0',
       'f6947f54', '6c930e6e', '6088b756', '9ed8f6da', '3edf6747', '53c6e11a',
       'a1192f43', '4d6737eb', 'd2659ab4', 'd38c2fd7', '3bb91ced', 'd88ca108',
       'a76029ee', '38074c54', '86ba578b', '0d18d96c']

misses_id = ['08fd73f3', '56817e2b', '37c53127', '3323d7e9', 'ca11f653', '1c178d24',
       '00c73085', '36fa3ebe', '16dffff1', '895865f3', '28520915', 'b012cd7f',
       'f5b8c21a', 'b74258a0', 'f6947f54', '6c930e6e', '38074c54']

activity_list = ['Bottle Filler (Activity)', 'Sandcastle Builder (Activity)',
       'Chicken Balancer (Activity)', 'Fireworks (Activity)',
       'Flower Waterer (Activity)', 'Bug Measurer (Activity)',
       'Egg Dropper (Activity)', 'Watering Hole (Activity)']

list_4020_4030 = {'Bottle Filler (Activity)':[4020,4030],
                 'Bug Measurer (Activity)':[4030],
                 'Chicken Balancer (Activity)':[4020,4030],
                 'Egg Dropper (Activity)':[4020],
                 'Fireworks (Activity)':[4020,4030],
                 'Flower Waterer (Activity)':[4020,4030],
                 'Sandcastle Builder (Activity)':[4020,4030],
                 'Watering Hole (Activity)':[4020]}

game_list = ['Chow Time', 'Scrub-A-Dub', 'All Star Sorting', 'Dino Drink',
       'Bubble Bath', 'Crystals Rule', 'Dino Dive', 'Pan Balance',
       'Happy Camel', 'Air Show', 'Leaf Leader']

def get_data(user_sample,test_set = False):
    

    duration_id_list = {eve:0 for eve in duration_event_id}
    duration_id_count = {eve:0 for eve in duration_event_id}
    clip_abandon = 0
    
    activity_abandon_dict = {eve:0 for eve in activity_list}
    activity_abandon_cnt = {eve:0 for eve in activity_list}


    
    
    #### round
    num_round_list = []
    num_round_max_list = []

    round_title_max_dict = {'round_Scrub-A-Dub_max':[], 
                         'round_Dino Drink_max':[], 
                         'round_All Star Sorting_max':[], 
                         'round_Air Show_max':[],
                        'round_Crystals Rule_max':[],
                         'round_Bubble Bath_max':[], 
                         'round_Bottle Filler (Activity)_max':[],
                        'round_Dino Dive_max':[], 
                         'round_Chow Time_max':[], 
                         'round_Pan Balance_max':[], 
                         'round_Happy Camel_max':[],
                        'round_Leaf Leader_max':[]}    
    
    #### misss
    num_miss = []
    miss_id_list = {eve:[] for eve in misses_id}
    miss_37c53127_dict = {'123':[],'456':[],'789':[],'other_level':[]}
    level_37c53127_list = []
    level_37c53127_list_max = []
    
    
    
    ###加时间
    count_game_diff_assessment = 0
    count_assessment_diff_assessment = 0 
    count_activity_diff_assessment = 0 
    count_clip_diff_assessment = 0
    
    last_world = -1
    ###1是计算是不是有直接开始assessment
    shangyici_world_time1 = {'syc_MAGMAPEAK':[],'syc_TREETOPCITY':[],'syc_CRYSTALCAVES':[]}
    shangyici_world_time2 = {'syc_MAGMAPEAK':[],'syc_TREETOPCITY':[],'syc_CRYSTALCAVES':[]}
    shangyici_test_time = {'syc_Mushroom Sorter (Assessment)':[],'syc_Chest Sorter (Assessment)':[],'syc_Bird Measurer (Assessment)':[],
                           'syc_Cauldron Filler (Assessment)':[],'syc_Cart Balancer (Assessment)':[]}
    #accumulated_actions = 0
    time_in_session = -1
    durations = []
    all_times = 0
    count_type_dict = {'count_Game':0,'count_Assessment':0,'count_Activity':0,'count_Clip':0}
    count_world_dict = {'count_MAGMAPEAK':0,'count_TREETOPCITY':0,'count_CRYSTALCAVES':0}
    time_type_dict = {'time_Game':0,'time_Assessment':0,'time_Activity':0,'time_Clip':0}
    time_world_dict = {'time_MAGMAPEAK':0,'time_TREETOPCITY':0,'time_CRYSTALCAVES':0}
    count_session = 0
    num_th_assess = 0
    all_features = []
    child_id = user_sample.installation_id.values[0]
    accuracy_groups = {0:0, 1:0, 2:0, 3:0}
    abandon_times = 0
    game_abandon = 0
    games_all = 0##game_all 会统一统计 可以删去
    num_game_correct = 0
    num_game_false = 0
    accum_avg_acc=[]
    accum_avg_group = []
    times_game_no_success = 0
    times_game_success = 0
    game_correct_list = []##acc_list
    num_enter_game = 0
    enter_game_acc_list = []
    enter_game_num_corr = 0
    enter_game_num_false = 0
    grp_assess_list = {'grp_Bird Measurer (Assessment)':[],'grp_Chest Sorter (Assessment)':[],\
                              'grp_Cauldron Filler (Assessment)':[],'grp_Cart Balancer (Assessment)':[],\
                              'grp_Mushroom Sorter (Assessment)':[]}
    acc_assess_list = {'acc_Bird Measurer (Assessment)':[],'acc_Chest Sorter (Assessment)':[],\
                              'acc_Cauldron Filler (Assessment)':[],'acc_Cart Balancer (Assessment)':[],\
                              'acc_Mushroom Sorter (Assessment)':[]}
    curr_world_type_time={'time_Game_curr_world':0,'time_Assessment_curr_world':0,'time_Activity_curr_world':0,'time_Clip_curr_world':0}
    curr_world_type_cnt = {'count_Game_curr_world':0,'count_Assessment_curr_world':0,'count_Activity_curr_world':0,'count_Clip_curr_world':0}
    curr_world_type_time_all = 0
    curr_world_type_count_all = 0
    event_code_count = {ev: 0 for ev in list_of_event_code}
    event_id_count = {eve: 0 for eve in list_of_event_id}
    title_count = {eve: 0 for eve in list_title}
    title_event_code_count = {t_eve: 0 for t_eve in list_title_event_code}

    
    for i,session in user_sample.groupby('game_session',sort=False):
        session_type = session.type.values[0]
        session_title = session.title.values[0]
        session_world = session.world.values[0]
        session_time_stamp_min = session.timestamp.min()
        session_time_stamp_max = session.timestamp.max()
        
        if session_title == 'Welcome to Lost Lagoon!':
            num_enter_game +=1
            enter_game_acc_list = []
            enter_game_num_corr = 0
            enter_game_num_false = 0
        
        if session_type == 'Game':
            games_all +=1
            game_assess_df = session[session.event_data.str.contains('"correct":')].copy()
            if game_assess_df.shape[0] >0:
                num_corr = np.sum(session.event_data.str.contains('"correct":true').astype(int))
                num_false = np.sum(session.event_data.str.contains('"correct":false').astype(int))
                num_game_correct += num_corr
                num_game_false += num_false
                enter_game_num_corr += num_corr
                enter_game_num_false +=num_false
                game_correct_list.append(num_corr/(num_corr + num_false))
                enter_game_acc_list.append(num_corr/(num_corr + num_false))
                if num_false>0 and num_corr==0:
                    times_game_no_success +=1
                if num_corr >0:
                    times_game_success +=1
            else:
                game_abandon +=1
             
        
        if session_type == 'Assessment':
            session_attempt = session[((session.event_code==4100)&(session.title != 'Bird Measurer (Assessment)'))\
              |((session.event_code==4110)&(session.title == 'Bird Measurer (Assessment)'))]
            mark = (test_set & (len(session)==1) & (session.event_code.values[0]==2000))
            
            
        

            if (session_attempt.shape[0]>0)|mark:
                shangyici_test_time['syc_'+session_title].append(session_time_stamp_min)
                shangyici_world_time1['syc_'+session_world].append(session_time_stamp_min)
                features = dict()
                ####miss
                features['sum_num_miss'] = np.sum(num_miss) if len(num_miss)!=0 else -1  
                features['mean_num_miss'] = np.mean(num_miss) if len(num_miss)!=0 else -1
                temp_dict = {'mean_num_miss'+key:np.mean(values) if len(values)!=0 else -1 for key,values in miss_id_list.items()}
                features.update(temp_dict.copy())
                temp_dict = {'sum_num_miss'+key:np.sum(values) if len(values)!=0 else -1 for key,values in miss_id_list.items()}
                features.update(temp_dict.copy())
                temp_dict = {'last_5_sum_num_miss'+key:np.sum(values[-5:]) if len(values)!=0 else -1 for key,values in miss_id_list.items()}
                features.update(temp_dict.copy())
#                 temp_dict = {'mean_num_miss_level_37c53127'+key:np.mean(values) if len(values)!=0 else -1 for key,values in miss_37c53127_dict.items()}
#                 features.update(temp_dict)
#                 temp_dict = {'sum_num_miss_level_37c53127'+key:np.sum(values) if len(values)!=0 else -1 for key,values in miss_37c53127_dict.items()}
#                 features.update(temp_dict)
#                 temp_dict = {'length_num_miss_level_37c53127'+key:len(values) for key,values in miss_37c53127_dict.items()}
#                 features.update(temp_dict)
                features['mean_level_37c53127'] = np.mean(level_37c53127_list) if len(level_37c53127_list)!=0 else -1
                features['mean_level_37c53127_max'] = np.mean(level_37c53127_list_max) if len(level_37c53127_list_max)!=0 else -1
#                 features['sum_level_37c53127_max'] = np.sum(level_37c53127_list_max) if len(level_37c53127_list_max)!=0 else -1
                
                #####

                temp_dict = {'overall_mean_'+key: values/duration_id_count[key] if duration_id_count[key]!=0 else -1 for key,values in duration_id_list.items()}
                features.update(temp_dict.copy())
                temp_dict = {'overall_sum_'+key:values for key,values in duration_id_list.items()}
                features.update(temp_dict.copy())

                
                
                features['num_round_list'] = np.mean(num_round_list) if len(num_round_list)!=0 else -1
                features['num_round_max_list'] = np.mean(num_round_max_list) if len(num_round_max_list)!=0 else -1
                features['num_round_max_sum'] = np.sum(num_round_max_list) if len(num_round_max_list)!=0 else -1
                features['num_round_max_std'] = np.std(num_round_max_list) if len(num_round_max_list)!=0 else -1


                temp_round_max_title = {'mean1_'+key:np.mean(value) if len(value)!=0 else -1 for key,value in round_title_max_dict.items()}
                features.update(temp_round_max_title.copy())
                temp_round_max_title = {'sum1_'+key:np.sum(value) if len(value)!=0 else -1 for key,value in round_title_max_dict.items()}
                features.update(temp_round_max_title.copy())
                
                #features['accumulated_actions'] = accumulated_actions
                features['last_session_time'] = time_in_session
                #features['all_times']= all_times
                features['duration_mean'] = np.mean(durations) if len(durations)!=0 else -1
                features['duration_std'] = np.std(durations) if len(durations)!=0 else -1
                features['mean_game_acc'] = np.mean(game_correct_list) if len(game_correct_list)!=0 else -1
                features['std_game_acc'] = np.std(game_correct_list) if len(game_correct_list)!=0 else -1
                features['mean_enter_game_acc'] = np.mean(enter_game_acc_list) if len(enter_game_acc_list)!=0 else -1
                features['std_enter_game_acc'] = np.std(enter_game_acc_list) if len(enter_game_acc_list)!=0 else -1
                #features['last_5_mean_game_acc'] = np.mean(game_correct_list[-5:]) if len(game_correct_list)!=0 else -1
                features['last_1_mean_game_acc'] = game_correct_list[-1] if len(game_correct_list)!=0 else -1
                features['count_game_acc'] = len(game_correct_list)
                features['num_enter_game'] = num_enter_game
                features.update(event_code_count.copy())
                features.update(event_id_count.copy())
                features.update(title_count.copy())
                features.update(title_event_code_count.copy())
                features.update(count_type_dict.copy())
                features.update(count_world_dict.copy())
                features['clip_abandon'] = clip_abandon
                features['clip_abandon_ratio'] = clip_abandon/features['count_Clip'] if features['count_Clip']!=0 else -1

                

                
                temp_dict = {'abandon_ratio_'+eve:values/activity_abandon_cnt[eve] if activity_abandon_cnt[eve]!=0 else -1 \
                             for eve,values in activity_abandon_dict.items()}
                features.update(temp_dict.copy())
                
                temp_dict = {'abandon_cnt_'+eve:values for eve,values in activity_abandon_dict.items()}
                features.update(temp_dict.copy())
                
                
                if session_world!='NONE':
                    features['curr_world_time'] = time_world_dict['time_'+session_world]
                    #features['curr_world_time_per'] = time_world_dict['time_'+session_world]/all_times if all_times!=0 else -1
                features['count_session'] = count_session
                features.update(time_type_dict.copy())
                features.update(time_world_dict.copy())
                
                temp_acc_assess_list = {key: np.mean(values) if len(values)!=0 else -1 for key,values in acc_assess_list.items()}
                features.update(temp_acc_assess_list.copy())


                #features['times_game_no_success'] = times_game_no_success
                #features['times_game_success'] = times_game_success
                features['ratio_time_game_success'] = times_game_success/(times_game_success + times_game_no_success) if (times_game_success + times_game_no_success)!=0 else -1
                features.update(accuracy_groups.copy())
                temp_all = accuracy_groups[0] + accuracy_groups[1] + accuracy_groups[2] + accuracy_groups[3]
                if temp_all==0:
                    features['0_ratio'] = 0
                    features['1_ratio'] = 0
                    features['2_ratio'] = 0
                    features['3_ratio'] = 0
                else:
                    features['0_ratio'] = accuracy_groups[0]/temp_all
                    features['1_ratio'] = accuracy_groups[1]/temp_all
                    features['2_ratio'] = accuracy_groups[2]/temp_all
                    features['3_ratio'] = accuracy_groups[3] /temp_all  
                features['installation_id'] = child_id
                features['game_session'] = i
                features['title'] = session_title
                features['world'] = session_world
                features['num_th_assess'] = num_th_assess
                features['abandon_times'] = abandon_times
                num_th_assess += 1
                features['accum_avg_acc'] = np.mean(accum_avg_acc) if len(accum_avg_acc)!=0 else -1
                features['accum_avg_group'] = np.mean(accum_avg_group) if len(accum_avg_group)!=0 else -1
                features['accum_std_acc'] = np.std(accum_avg_acc) if len(accum_avg_acc)!=0 else -1
                features['accum_std_group'] = np.std(accum_avg_group) if len(accum_avg_group)!=0 else -1

                features['last_5_accum_avg_acc'] = np.mean(accum_avg_acc[-5:]) if len(accum_avg_acc)!=0 else -1
                #features['last_5_accum_avg_group'] = np.mean(accum_avg_group[-5:]) if len(accum_avg_group)!=0 else -1
                features['last_5_accum_std_acc'] = np.std(accum_avg_acc[-5:]) if len(accum_avg_acc)!=0 else -1
                features['last_5_accum_std_group'] = np.std(accum_avg_group[-5:]) if len(accum_avg_group)!=0 else -1
                #features['last_1_accum_avg_acc'] = accum_avg_acc[-1] if len(accum_avg_acc)!=0 else -1
                features['last_1_accum_avg_group'] = accum_avg_group[-1] if len(accum_avg_group)!=0 else -1
                ###加这两个的count
                features['overall_game_corr_ratio'] = num_game_correct/(num_game_correct + num_game_false) if (num_game_correct + num_game_false)!=0 else -1
                features['enter_game_overall_game_corr_ratio'] = enter_game_num_corr/(enter_game_num_corr + enter_game_num_false) if (enter_game_num_corr + enter_game_num_false) !=0 else -1

                
                features['game_abandon'] = game_abandon
                features['game_abandon_ratio'] = game_abandon/games_all if games_all!=0 else -1
                features['mean_same_assess_grp'] = np.mean(grp_assess_list['grp_'+session_title]) if len(grp_assess_list['grp_'+session_title])!=0 else -1
                features['mean_same_assess_acc'] = np.mean(acc_assess_list['acc_'+session_title]) if len(acc_assess_list['acc_'+session_title])!=0 else -1
                features['std_same_assess_grp'] = np.std(grp_assess_list['grp_'+session_title]) if len(grp_assess_list['grp_'+session_title])!=0 else -1
                features['std_same_assess_acc'] = np.std(acc_assess_list['acc_'+session_title]) if len(acc_assess_list['acc_'+session_title])!=0 else -1
                features['count_same_assess_grp'] = len(grp_assess_list['grp_'+session_title])

                features['last_3_mean_same_assess_grp'] = np.mean(grp_assess_list['grp_'+session_title][-3:]) if len(grp_assess_list['grp_'+session_title])!=0 else -1
                features['last_3_mean_same_assess_acc'] = np.mean(acc_assess_list['acc_'+session_title][-3:]) if len(acc_assess_list['acc_'+session_title])!=0 else -1
                
                features['last_1_mean_same_assess_grp'] = grp_assess_list['grp_'+session_title][-1] if len(grp_assess_list['grp_'+session_title])!=0 else -1
                features['last_1_mean_same_assess_acc'] = acc_assess_list['acc_'+session_title][-1] if len(acc_assess_list['acc_'+session_title])!=0 else -1
                
                features['last_test_to_now_time'] = (shangyici_test_time['syc_'+session_title][-1] \
                                                     - shangyici_test_time['syc_'+session_title][-2]).total_seconds() if len(shangyici_test_time['syc_'+session_title])>=2 else -1
                features['last_world_to_now_time1'] =  (shangyici_world_time1['syc_'+session_world][-1]\
                                                        - shangyici_world_time1['syc_'+session_world][-2]).total_seconds() if len(shangyici_world_time1['syc_'+session_world])>=2 else -1
                features['last_world_to_now_time2'] =  (shangyici_world_time2['syc_'+session_world][-1]\
                                                        - shangyici_world_time2['syc_'+session_world][-2]).total_seconds() if len(shangyici_world_time2['syc_'+session_world])>=2 else -1
                features['count_game_diff_assessment'] = count_game_diff_assessment
                features['count_assessment_diff_assessment'] = count_assessment_diff_assessment
                features['count_activity_diff_assessment'] = count_activity_diff_assessment
                features['count_clip_diff_assessment'] = count_clip_diff_assessment
                features.update(curr_world_type_time.copy())
                features.update(curr_world_type_cnt.copy())

                
                
                
#                 features['curr_world_type_time_all'] = curr_world_type_time_all
#                 features['curr_world_type_count_all'] = curr_world_type_count_all
                if not mark:
                    true_attempt = np.sum(session_attempt.event_data.str.contains('"correct":true').astype(int))
                    false_attempt = np.sum(session_attempt.event_data.str.contains('"correct":false').astype(int))
                    features['accuracy'] = true_attempt/(true_attempt + false_attempt) if (true_attempt + false_attempt)!=0 else 0
                    features['accuracy_group'] = to_accuracy_group(features['accuracy'])
                    accum_avg_acc.append(features['accuracy'])
                    accum_avg_group.append(features['accuracy_group'])
                    accuracy_groups[features['accuracy_group']] += 1
                    grp_assess_list['grp_'+session_title].append(features['accuracy_group'])
                    acc_assess_list['acc_'+session_title].append(features['accuracy'])
                all_features.append(features)
                count_game_diff_assessment = 0
                count_assessment_diff_assessment = 0
                count_activity_diff_assessment = 0
                count_clip_diff_assessment = 0
            else:
                abandon_times +=1
            
        def update_counters(counter, col):
            num_of_session_count = Counter(session[col])
            for k in num_of_session_count.keys():
                x = k
                counter[x] += num_of_session_count[k]
            return counter
        event_code_count = update_counters(event_code_count, "event_code")
        event_id_count = update_counters(event_id_count, "event_id")
        title_count = update_counters(title_count, 'title')
        title_event_code_count = update_counters(title_event_code_count,'title_event_code')
        count_session +=1
        count_type_dict['count_'+session_type] +=1
        if session_type != 'Clip':
            time_in_session = (session_time_stamp_max - session_time_stamp_min).total_seconds()
        else:
            try:
                temp_time = session.timestamp.min()
                temp_index = session.index[0]+1
                time_in_session = ((user_sample.loc[temp_index,'timestamp']) - temp_time).total_seconds()
                if time_in_session >clip_time[session_title]:
                    time_in_session = clip_time[session_title]
            except:
                time_in_session = clip_time[session_title]
            if time_in_session < 0.5*clip_time[session_title]:
                clip_abandon +=1
        time_type_dict['time_'+session_type] += time_in_session
        durations.append(time_in_session)
        all_times +=time_in_session
        if session_world!='NONE':
            count_world_dict['count_'+session_world] +=1
            time_world_dict['time_'+session_world] += time_in_session
        #accumulated_actions += len(session)
        
        if session_world!=last_world:
            curr_world_type_time={'time_Game_curr_world':0,'time_Assessment_curr_world':0,'time_Activity_curr_world':0,'time_Clip_curr_world':0}
            curr_world_type_cnt = {'count_Game_curr_world':0,'count_Assessment_curr_world':0,'count_Activity_curr_world':0,'count_Clip_curr_world':0}
#             curr_world_type_time_all = 0
#             curr_world_type_count_all = 0
            if session_world != 'NONE':
                shangyici_world_time2['syc_'+session_world].append(session_time_stamp_min)
            last_world = session_world
        if session_world != 'NONE':
#             curr_world_type_time_all +=time_in_session
#             curr_world_type_count_all +=1
            curr_world_type_time['time_'+session_type+'_curr_world'] +=time_in_session
            curr_world_type_cnt['count_'+session_type+'_curr_world'] +=1
        #####1230 count4070系列
        
        if session_type == 'Game':
            count_game_diff_assessment +=1
        if session_type == 'Activity':
            count_activity_diff_assessment+=1
        if session_type == 'Assessment':
            count_assessment_diff_assessment+=1
        if session_type =='Clip':
            count_clip_diff_assessment +=1
        session_miss = session[session.event_data.str.contains('"misses":')].copy()
        if session_miss.shape[0]>0:
            session_miss['num_miss'] = session_miss.event_data.map(lambda x:json.loads(x)['misses'])
            for id1,miss in zip(session_miss['event_id'],session_miss['num_miss']):
                num_miss.append(miss)
                miss_id_list[id1].append(miss)
            session_37c53127 = session_miss[session_miss.event_id=='37c53127']
            if session_37c53127.shape[0]>0:
                session_37c53127['level'] = session_37c53127.event_data.map(lambda x:json.loads(x)['level'])
                level_37c53127_list_max.append(np.max(session_37c53127['level']))
                for level,miss in zip(session_37c53127['level'],session_37c53127['num_miss']):
                    level_37c53127_list.append(level)
#                     if level in [1,2,3]:
#                         miss_37c53127_dict['123'].append(miss)
#                     elif level in [4,5,6]:
#                         miss_37c53127_dict['456'].append(miss)
#                     elif level in [7,8,9]:
#                         miss_37c53127_dict['789'].append(miss)
#                     else:
#                         miss_37c53127_dict['other_level'].append(miss)
                    

        ####round
        session_round = session[(session.event_data.str.contains('"round":'))&(session.event_code==2030)].copy()
        if session_round.shape[0]>0:
            session_round = session[session.event_data.str.contains('"round":')].copy()
            session_round['num_round'] = session_round.event_data.map(lambda x:json.loads(x)['round'])
            num_round_list.extend(list(session_round.num_round))
            num_round_max_list.append(np.max(session_round.num_round))
            round_title_max_dict['round_'+session_title+'_max'].append(np.max(session_round.num_round))
        if np.sum(session.event_data.str.contains('"duration":'))>0:
            session_duration = session[(session.event_data.str.contains('"duration":'))&(session.event_id.map(lambda x:x in duration_event_id))].copy()
            if session_duration.shape[0]>0:
                session_duration['duration'] = session_duration.event_data.map(lambda x:json.loads(x)['duration'])
                for id1,duration in zip(session_duration['event_id'],session_duration['duration']):

                    duration_id_list[id1] += duration
                    duration_id_count[id1] += 1
        if session_type=='Activity':
            activity_abandon_cnt[session_title]+=1
            if session_title in ['Bottle Filler (Activity)','Chicken Balancer (Activity)',
                                'Fireworks (Activity)','Flower Waterer (Activity)','Sandcastle Builder (Activity)']:
                num_4020 = np.sum(session.event_code==4020)
                num_4030 = np.sum(session.event_code==4030)
                if num_4020==0 or num_4030 ==0:
                    activity_abandon_dict[session_title]+=1
            elif session_title in ['Bug Measurer (Activity)']:
                num_4030 = np.sum(session.event_code==4030)
                if num_4030 ==0:
                    activity_abandon_dict[session_title]+=1
            elif session_title in ['Egg Dropper (Activity)','Watering Hole (Activity)']:
                num_4020 = np.sum(session.event_code==4020)
                if num_4020 ==0:
                    activity_abandon_dict[session_title]+=1



    if test_set:
        return all_features[-1]    
    return all_features

In [17]:
def get_train(train):
    train_all = []
    for ins_id,user_sample in tqdm(train.groupby('installation_id',sort=False)):
        train_all.extend(get_data(user_sample))
    return pd.DataFrame(train_all)
train_all=get_train(train)

100%|██████████| 17000/17000 [54:43<00:00,  5.18it/s]


In [18]:
def get_train(train):
    train_all = []
    for ins_id,user_sample in tqdm(train.groupby('installation_id',sort=False)):
        train_all.extend(get_data(user_sample))
    return pd.DataFrame(train_all)
train_all_test=get_train(test)

100%|██████████| 1000/1000 [05:20<00:00,  3.12it/s]


In [19]:
train_all = train_all.append(train_all_test,ignore_index=True)

In [20]:
def get_test(test):
    test_all = []
    for ins_id,user_sample in tqdm(test.groupby('installation_id',sort=False)):
        test_all.append(get_data(user_sample,test_set = True))
    return pd.DataFrame(test_all)

test_all = get_test(test)

100%|██████████| 1000/1000 [05:22<00:00,  3.10it/s]


In [21]:
y_train = train_all.accuracy_group.copy()
drop_col = ['accuracy','game_session','accuracy_group','installation_id']
ins_id = train_all.installation_id
X_train = train_all.drop(drop_col,axis=1)
X_test = test_all.drop(['game_session','installation_id'],axis=1)
X_train.columns = X_train.columns.astype(str)
X_test.columns = X_test.columns.astype(str)
X_train = X_train[sorted(X_train.columns.tolist())]
X_test = X_test[sorted(X_test.columns.tolist())]

In [22]:
for f in X_train.columns:
    if (X_train[f].dtype=='object') or (X_test[f].dtype=='object'): 
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(X_train[f])+list(X_test[f]))
        X_train[f] = lbl.transform(list(X_train[f]))
        X_test[f] = lbl.transform(list(X_test[f]))

In [23]:
all_features = X_train.columns.tolist()
num_features = all_features.copy()
num_features.remove('world')
num_features.remove('title')
cat_features = ['world','title']

#### choose top n

In [24]:
X_train.drop(orders[700:],axis=1,inplace = True)
X_test.drop(orders[700:],axis=1,inplace = True)
print(X_train.shape)

(19708, 700)


In [25]:
print(X_train.shape)
missing_boolen_list = []
for _ in num_features:
    if _ in X_train.columns:
        if X_train[_].min()==-1:
            X_train[_+'missing_boolen'] = list(np.where(X_train[_]==-1,1,0))
            X_test[_+'missing_boolen'] = list(np.where(X_test[_]==-1,1,0))
            missing_boolen_list.append(_+'missing_boolen')
print(X_train.shape)

(19708, 700)
(19708, 873)


In [26]:
X_train.replace(-1,0,inplace=True)
X_test.replace(-1,0,inplace=True)

In [27]:
num_features1 = list(set(num_features).intersection(X_train.columns))

In [28]:

X_train[num_features1] = np.log1p(X_train[num_features1])
X_test[num_features1] = np.log1p(X_test[num_features1])

In [29]:
from sklearn.preprocessing import StandardScaler,OneHotEncoder

In [30]:
X_train_num = X_train[num_features1].copy()
X_test_num = X_test[num_features1].copy()
X_train_cat = X_train[cat_features].copy()
X_test_cat = X_test[cat_features].copy()

In [31]:
sds = StandardScaler()
sds.fit(X_train_num)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [32]:
X_train_num_ted = pd.DataFrame(sds.transform(X_train_num),columns=num_features1)
X_test_num_ted = pd.DataFrame(sds.transform(X_test_num),columns=num_features1)

In [33]:
one_hot = OneHotEncoder(sparse=False)
one_hot.fit(X_train_cat)
X_train_cat_ted = pd.DataFrame(one_hot.transform(X_train_cat),columns=one_hot.get_feature_names(['world','title']))
X_test_cat_ted = pd.DataFrame(one_hot.transform(X_test_cat),columns=one_hot.get_feature_names(['world','title']))

In [34]:
X_train = pd.concat([X_train_num_ted,X_train_cat_ted,X_train[missing_boolen_list]],axis=1)
X_test = pd.concat([X_test_num_ted,X_test_cat_ted,X_test[missing_boolen_list]],axis=1)

In [35]:
print(X_train.shape)

(19708, 879)


In [36]:
# debug for "LightGBMError: Do not support special JSON characters in feature name."

X_train.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in X_train.columns]
X_test.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in X_test.columns]

In [37]:

print(X_train.shape)

(19708, 879)


In [38]:
def random1(ins_id,sample_size):
    list1 = []
    for id1 in ins_id.unique():
        temp_list = list(ins_id[ins_id == id1].index)
        list1.append(list(np.random.choice(temp_list,sample_size)))
    return np.array(list1)

truncated_groups = random1(ins_id,5000)

In [39]:
pred_lgb,impor1,oof_class_lgb,oof_num_lgb,pred_num1 = run_all_nn(X_train,y_train,ins_id,X_test)
print(qwk(oof_class_lgb[X_train.num_th_assess ==0],y_train[X_train.num_th_assess ==0]))
print(qwk(oof_class_lgb[X_train.num_th_assess !=0],y_train[X_train.num_th_assess !=0]))

1 th seed begins----------------------
Train on 14781 samples, validate on 4927 samples
Epoch 1/300
 - 4s - loss: 5.7065 - val_loss: 3.3368
Epoch 2/300
 - 2s - loss: 3.7312 - val_loss: 2.0891
Epoch 3/300
 - 2s - loss: 2.8472 - val_loss: 1.4752
Epoch 4/300
 - 2s - loss: 2.3551 - val_loss: 1.1746
Epoch 5/300
 - 2s - loss: 2.1022 - val_loss: 1.1365
Epoch 6/300
 - 2s - loss: 1.9336 - val_loss: 1.0789
Epoch 7/300
 - 2s - loss: 1.8073 - val_loss: 1.0352
Epoch 8/300
 - 2s - loss: 1.6713 - val_loss: 1.0439
Epoch 9/300
 - 2s - loss: 1.6121 - val_loss: 1.0378
Epoch 10/300
 - 2s - loss: 1.5450 - val_loss: 1.0342
Epoch 11/300
 - 2s - loss: 1.4742 - val_loss: 1.0086
Epoch 12/300
 - 2s - loss: 1.4226 - val_loss: 1.0241
Epoch 13/300
 - 2s - loss: 1.3871 - val_loss: 1.0179
Epoch 14/300
 - 2s - loss: 1.3333 - val_loss: 1.0435
Epoch 15/300
 - 2s - loss: 1.2922 - val_loss: 0.9989
Epoch 16/300
 - 2s - loss: 1.2680 - val_loss: 0.9819
Epoch 17/300
 - 2s - loss: 1.2349 - val_loss: 0.9942
Epoch 18/300
 - 2s -

ZeroDivisionError: float division by zero

In [40]:
sample_submission['accuracy_group'] = pred_lgb.astype(int)
sample_submission.to_csv('./submission.csv', index=False)
sample_submission['accuracy_group'].value_counts(normalize=True)

3    0.484
2    0.202
1    0.160
0    0.154
Name: accuracy_group, dtype: float64